In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import requests
import folium
import json
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

print("Libraries imported OK")

Libraries imported OK


The first thing to do is to load my credentials for using the Foursquare API.

In [2]:
CLIENT_ID = 'IMXET33FHBIMXV5PPPWLFV5U5WRR30T352CQ1440KDXVIASU' 
CLIENT_SECRET = 'O5TQUCYRKMBHYUXMAZT5MLZ3GLMRIDRD2CRIKNTTNR5ED2ES' 
VERSION = '20200424'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IMXET33FHBIMXV5PPPWLFV5U5WRR30T352CQ1440KDXVIASU
CLIENT_SECRET:O5TQUCYRKMBHYUXMAZT5MLZ3GLMRIDRD2CRIKNTTNR5ED2ES


Next, I'll get the location of Buenos Aires in order to create a map.

In [3]:
address = 'Buenos Aires, AR'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude_BA = location.latitude
longitude_BA = location.longitude

print('Buenos Aires cordinates are: {}, {}'.format(latitude_BA, longitude_BA))

Buenos Aires cordinates are: -34.6075682, -58.4370894


Create a map for checking that the location is correct.

In [4]:
BA_map = folium.Map(location = [latitude_BA, longitude_BA], zoom_start = 12)
BA_map

Make the request to the API for getting the data needed.

In [5]:
search_query = 'Food'
radius = 14000
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_BA, longitude_BA, VERSION, search_query, radius, LIMIT)
url

Food .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=IMXET33FHBIMXV5PPPWLFV5U5WRR30T352CQ1440KDXVIASU&client_secret=O5TQUCYRKMBHYUXMAZT5MLZ3GLMRIDRD2CRIKNTTNR5ED2ES&ll=-34.6075682,-58.4370894&v=20200424&query=Food&radius=14000&limit=100'

Show the results.

In [ ]:
results = requests.get(url).json()
results

Clear the data, drop some columns and rename.

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
df = json_normalize(venues)

# This columns are not necessary
columns_to_delete = ['id', 'categories', 'referralId', 'hasPerk', 'location.labeledLatLngs', 
                    'location.distance', 'location.cc', 'location.city', 'location.state', 
                    'location.country', 'location.formattedAddress', 'location.address',
                    'location.postalCode', 'location.crossStreet', 'location.neighborhood',
                    'venuePage.id']

# Drop the columns
df.drop(columns = columns_to_delete, inplace = True)

# Rename the columns
df.rename(columns = {'name':'Name','location.lat':'Latitude', 'location.lng':'Longitude'}, inplace = True)

df.head()

,Name,Latitude,Longitude
0,Green Factory - Food Truck,-34.596250,-58.455984
1,Food shop,-34.608937,-58.441043
2,Food Land,-34.609578,-58.430324
3,Green Dely-natural food,-34.610639,-58.429007
4,Green House - Natural food,-34.614999,-58.430035


Data is ready. Now I show the map with de data.

In [8]:
BA_food_map = folium.Map(location = [latitude_BA, longitude_BA], zoom_start = 12)

i = 0
for place in df.loc[:,'Name']:
    folium.Marker([df.loc[i, 'Latitude'], df.loc[i, 'Longitude']]).add_to(BA_food_map)
    i += 1

BA_food_map

Now the locations must be clustered. Run the k-means algorithm.

In [9]:
# set number of clusters
kclusters = 4

BA_clustering = df.drop('Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(BA_clustering)

df.insert(0, 'Cluster Labels', kmeans.labels_)
df.head()

,Cluster Labels,Name,Latitude,Longitude
0,0,Green Factory - Food Truck,-34.596250,-58.455984
1,0,Food shop,-34.608937,-58.441043
2,0,Food Land,-34.609578,-58.430324
3,0,Green Dely-natural food,-34.610639,-58.429007
4,0,Green House - Natural food,-34.614999,-58.430035


Define a function for plotting the groups in different colours.

In [10]:
def getcolor(i):
    if df.loc[i, 'Cluster Labels'] == 0:
        return 'red'
    elif df.loc[i, 'Cluster Labels'] == 1:
        return 'blue'
    elif df.loc[i, 'Cluster Labels'] == 2:
        return 'green'
    elif df.loc[i, 'Cluster Labels'] == 3:
        return 'yellow'

Show the map with the data clustered

In [11]:
BA_food_map = folium.Map(location = [latitude_BA, longitude_BA], zoom_start = 12)

i = 0
    
for p_code in df.loc[:, 'Name']:
    folium.CircleMarker(
        [df.loc[i, 'Latitude'], df.loc[i, 'Longitude']],
        radius=5,
        color=getcolor(i),
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(BA_food_map)
    i += 1

BA_food_map

Separate the clusters in different dataframes

In [12]:
cluster_1 = df[df['Cluster Labels'] == 0]
cluster_2 = df[df['Cluster Labels'] == 1]
cluster_3 = df[df['Cluster Labels'] == 2]
cluster_4 = df[df['Cluster Labels'] == 3]

Now I have to find the cluster with more points

In [13]:
cluster_lengths = {'cluster_1':cluster_1.shape[0], 'cluster_2':cluster_2.shape[0],
                  'cluster_3':cluster_3.shape[0], 'cluster_4':cluster_4.shape[0]}

largest_cluster = max(cluster_lengths, key = cluster_lengths.get)

if largest_cluster == 'cluster_1':
    cluster = 0;
elif largest_cluster == 'cluster_2':
    cluster = 1
elif largest_cluster == 'cluster_3':
    cluster = 2
elif largest_cluster == 'cluster_4':
    cluster = 3
    
print('The restaurant should be placed within the zone of cluster {}'.format(cluster))

The restaurant should be placed within the zone of cluster 1


Create a variable wich the points of the cluster selected

In [14]:
final_cluster = df[df['Cluster Labels'] == cluster]
final_cluster.head()

,Cluster Labels,Name,Latitude,Longitude
8,1,Siete Fast Food,-34.597454,-58.423057
10,1,Health Food,-34.598587,-58.420433
11,1,Arabian Food,-34.601063,-58.418983
12,1,Kangoo Pet Food,-34.590377,-58.424155
13,1,Countdown Beer and Food,-34.589657,-58.429817


Reset the index and delete te index column

In [ ]:
final_cluster.reset_index(inplace = True)
final_cluster.drop(columns = 'index', inplace = True)

Create a map with the limits of the zone where the restaurant will be set

In [16]:
latitude_min = min(final_cluster['Latitude'])
latitude_max = max(final_cluster['Latitude'])
longitude_min = min(final_cluster['Longitude'])
longitude_max = max(final_cluster['Longitude'])

latitude_centroid = np.mean(final_cluster['Latitude'])
longitude_centroid = np.mean(final_cluster['Longitude'])

BA_cluster_map = folium.Map(location = [latitude_centroid, longitude_centroid], zoom_start = 14)

i = 0  
for place in final_cluster.loc[:, 'Name']:
    folium.CircleMarker(
        [final_cluster.loc[i, 'Latitude'], final_cluster.loc[i, 'Longitude']],
        radius=5,
        color="blue",
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(BA_cluster_map)
    i += 1

# Draw the lines
folium.PolyLine(([latitude_min, longitude_min],[latitude_min,longitude_max]), color="red", weight=2.5, opacity=1).add_to(BA_cluster_map)
folium.PolyLine(([latitude_min, longitude_min],[latitude_max,longitude_min]), color="red", weight=2.5, opacity=1).add_to(BA_cluster_map)
folium.PolyLine(([latitude_max, longitude_min],[latitude_max,longitude_max]), color="red", weight=2.5, opacity=1).add_to(BA_cluster_map)
folium.PolyLine(([latitude_min, longitude_max],[latitude_max,longitude_max]), color="red", weight=2.5, opacity=1).add_to(BA_cluster_map)

BA_cluster_map

I have to select the point belonging to the red area which is farthest from the other restaurants. So I wil define a set of points uniformly distributed over the area.

In [18]:
N = 10 # dimension of the dataset

points = []

temp_x = np.linspace(latitude_min, latitude_max, N)
temp_y = np.linspace(longitude_min, longitude_max, N)

for i in range(N-2):
    points.append([])
    for j in range(N-2):
        points[i].append((temp_x[j+1], temp_y[i+1]))

Show in the map the points to be evaluated in blue and the restaurants in yellow.

In [19]:
BA_cluster_map_edge = folium.Map(location = [latitude_centroid, longitude_centroid], zoom_start = 14)

for i in range(N-2):
    for j in range(N-2):
        folium.CircleMarker([points[i][j][0],points[i][j][1]],
                           radius=5,
                            color="blue",
                            fill=True,
                            fill_color='#3186cc',
                            fill_opacity=0.7,
                            parse_html=False).add_to(BA_cluster_map_edge)
    
i = 0
for place in final_cluster.loc[:, 'Name']:
    folium.CircleMarker(
        [final_cluster.loc[i, 'Latitude'], final_cluster.loc[i, 'Longitude']],
        radius=5,
        color="yellow",
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(BA_cluster_map_edge)
    i += 1
    
BA_cluster_map_edge

Note that the border of the zone was excluded since we do not want to set up the restaurant near the other clusters.

Now I define a dataframe which is a copy of the cluster but without the columns 'Cluster Label' and 'Name'.

In [20]:
r = final_cluster.copy()
r.drop(columns = ['Cluster Labels','Name'], inplace = True)
r.head()

,Latitude,Longitude
0,-34.597454,-58.423057
1,-34.598587,-58.420433
2,-34.601063,-58.418983
3,-34.590377,-58.424155
4,-34.589657,-58.429817


Define a function to calculate distance between points.

In [21]:
def dist(A, index_df, B):
    return ( (A.loc[index_df]['Latitude'] - B[0])**2 + (A.loc[index_df]['Longitude'] - B[1])**2)**0.5

Define a function for select the minimun distance from a list of distances.

In [22]:
def dist_min(A, B):
    lst_dist = []
    for i in range(A.shape[0]):
        lst_dist.append(dist(A,i,B))
    return min(lst_dist)

Create a list wich contains the minimun distance to othe restaurant for each evaluation point.

In [23]:
lst_of_distances = []

for i in range(8):
    lst_of_distances.append([])
    for j in range(8):
        lst_of_distances[i].append(dist_min(r,points[i][j]))

Select the best point which is the one with the largest minimun distance to other restaurant.

In [24]:
index_i = lst_of_distances.index(max(lst_of_distances))
index_j = max(lst_of_distances).index(max(max(lst_of_distances)))
best_point = points[index_i][index_j]

print('The best point has the cordinates: {}, {}'.format(best_point[0], best_point[1]))

The best point has the cordinates: -34.597286953883625, -58.43821036956921


Finally, show the best point where the restaurant must be placed.

In [25]:
BA_cluster_map_edge = folium.Map(location = [latitude_centroid, longitude_centroid], zoom_start = 14)

    
i = 0
for place in final_cluster.loc[:, 'Name']:
    folium.CircleMarker(
        [final_cluster.loc[i, 'Latitude'], final_cluster.loc[i, 'Longitude']],
        radius=5,
        color="yellow",
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(BA_cluster_map_edge)
    i += 1
    
folium.CircleMarker(
        [best_point[0], best_point[1]],
        radius=5,
        color="red",
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(BA_cluster_map_edge)

BA_cluster_map_edge